Scikit-learnを用いることで以下のような統計的処理を簡単に記述することができます。

*   Classfication(分類)
    *   対象がどのカテゴリーに属するかを判断する
    *   迷惑メール判断、画像認識
*   Regression(回帰)
    *   対象に関連する連続値の値を予想する
    *   薬の効果反応の予測、株価の予測
*   Clustering(クラスタリング)
    *   似た対象のグルーピングを自動化する
    *   顧客区分け、実験結果のグルーピング
*   Dimensionality reduction(次元削減)
    *   確率変数を削減して簡略化する
    *   可視化, 効率化
*   Model Selection(モデル選択)
    *   モデルを比較検討し選択する
    *   精度向上
*   Preprocessing(前処理)
    *   特徴抽出と正規化
    *   テキストのような入力データを機械学習で使えるようなデータに変換する、など 


scikit-learnには機械学習のためのアルゴリズムモデルがたくさん用意されていて、それらのことをestimator(推定器) といいます。
それぞれ推定器はデータをfit(学習)することができます。例として、RandomForestClassifier にシンプルなデータを学習させてみましょう。

In [1]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)
X = [[ 1,  2,  3],  # 2 サンプル, 特徴の数が3のデータ。
    [11, 12, 13]]
y = [0, 1]  # 教師データ。[1,2,3]のデータを0に、[11,12,13]のデータを1に対応付ける。
clf.fit(X, y)

RandomForestClassifier(random_state=0)

ここで、fitの引数は以下のようなものになります。

*   X: サンプルデータ。データは行:サンプル、列:属性データの行列になる。
*   y: 回帰課題では実数，分類課題では整数。通常1次元の配列であり，n番目の値がXのn番目のサンプルの数値に対応する。

In [2]:
# トレーニングデータの分類を予測する
clf.predict(X) # => array([0, 1])

# 未知の値の分類予測
clf.predict([[4, 5, 6], [14, 15, 16]]) # => array([0, 1])
clf.predict([[14, 15, 16],[4, 5, 6]]) # =>array([1, 0])
clf.predict([[14, 15, 16],[7,8,9],[7,6,5],[4, 5, 6]]) # => array([1, 1, 0, 0])

array([1, 1, 0, 0])






---



**データ整形と前処理**

In [3]:
# StandardScalerはデータを標準化(各列の平均が0に、分散が1になるような数値の整形)する処理
# 平均は各データを足して要素数で割った数値
# 分散は各データの平均値との差を二乗し、平均をとったもの
from sklearn.preprocessing import StandardScaler
X = [[0, 15],
     [1, -10]]
ss = StandardScaler().fit(X) # フィッティングさせる。
ss.transform(X) # => array([[-1.,  1.],[ 1., -1.]]) # 訓練データを標準化したもの
Y = [[2,5],[4,-12]] # 未知データ
ss.transform(Y) # => array([[ 3.  ,  0.2 ],[ 7.  , -1.16]])

array([[ 3.  ,  0.2 ],
       [ 7.  , -1.16]])

**パイプライン処理: 前処理と推定器を繋ぐ**

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# pipelineオブジェクトを作成する
pipe = make_pipeline(
    StandardScaler(), # 標準化用の前処理
    LogisticRegression() # ロジスティック回帰の推定器
)

# アヤメのデータをロードして、トレーニングデータとテストデータに分ける
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# パイプラインを学習させる
pipe.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [5]:
accuracy_score(pipe.predict(X_test), y_test)

0.9736842105263158

**モデル評価**: 
前項のアヤメの例のように、予測の妥当性を確認する目的で学習データを訓練用データとテストデータに分けて、学習モデルの妥当性を評価する方法があり、これを交差検証(クロスバリデーション)と呼びます。
scikit-learnではこの交差検証を行う方法も簡単に扱えるよう用意されています。

In [6]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

# 回帰モデル用のデータを生成する この場合はサンプル数1000、random性を排除してデータを作る
X, y = make_regression(n_samples=1000, random_state=0) 
lr = LinearRegression()

# デフォルトではデータを5分割してクロスバリデーションを行う。
# 詳しくはk-分割交差検証 ( https://ja.wikipedia.org/wiki/%E4%BA%A4%E5%B7%AE%E6%A4%9C%E8%A8%BC#k-%E5%88%86%E5%89%B2%E4%BA%A4%E5%B7%AE%E6%A4%9C%E8%A8%BC )
# を参照
result = cross_validate(lr, X, y)  
result['test_score']  # => array([1., 1., 1., 1., 1.]) 
# データが簡単なので交差検証の妥当性が高い

array([1., 1., 1., 1., 1.])

**自動パラメータ探索**

すべての推定器には学習時に設定するパラメータがあり、これをハイパーパラメータと呼びます。例えば、特徴量の特徴抽出方法やモデルの種類を指します。
推定器の応用性はいくつかのパラメータに強く依存します。例えば、RandomForestRegressor には，森の中の木の数を決める n_estimators パラメータと，各木の最大深さを決める max_depth パラメータがあります。これらのパラメータは手元のデータに依存するため、正確な値が不明な場合が非常に多いです。

scikit-learnは最適なパラメータの組み合わせを自動的に見つけるツールを提供します。以下の例では、RandomizedSearchCV オブジェクトを用いて、ランダムフォレストのパラメータをランダムに探索します。探索が終わると，RandomizedSearchCV は最適なパラメータ・セットでフィッティングされた RandomForestRegressor として振る舞います。


In [7]:
from sklearn.datasets import fetch_california_housing
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import randint

X, y = fetch_california_housing(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# 探索するパラメータの範囲を指定する
param_distributions = {'n_estimators': randint(1, 5),
                       'max_depth': randint(5, 10)}

# searchCV を作成して、データにフィッティングしましょう。
search = RandomizedSearchCV(estimator=RandomForestRegressor(random_state=0),
                            n_iter=5,
                            param_distributions=param_distributions,
                            random_state=0)
search.fit(X_train, y_train)

# => RandomizedSearchCV(
#      estimator=RandomForestRegressor(random_state=0), 
#      n_iter=5, 
#      aram_distributions={'max_depth': ..., 'n_estimators': ...},
#      random_state=0
# )

search.best_params_ # => {'max_depth': 9, 'n_estimators': 4}

# これで探索オブジェクトは norman random forest estimatorとして振る舞うようになった
# max_depth = 9, n_estimators=4

# テスト用データとその教師データにたいして適用する
search.score(X_test, y_test) # => 0.735363411343253
# 比較的高い精度であることがわかる

0.735363411343253